In [162]:
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from skimage import io, transform, color
from skimage.feature import hog
from skimage import exposure
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import os
from tqdm import tqdm
import cv2

import shutil

### downlaod the data from kaggke

In [2]:


!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [3]:
!kaggle competitions download -c umor-classification

umor-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!unzip umor-classification.zip

Archive:  umor-classification.zip
replace images/images/1006.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

***

### read the image path

In [2]:
# # Load and preprocess training images
image_folder   = '/content/images/images/'

In [3]:
print(f"There are {len(os.listdir(image_folder ))} images in  dataset")

There are 775 images in  dataset


***

### read the csv files

In [4]:
train_df  = pd.read_csv('trainset.csv')
test_df   = pd.read_csv('testset.csv')

In [5]:
train_df['Class'].value_counts()

Benign       300
Malignant    148
Normal        91
Name: Class, dtype: int64

In [6]:
train_df

,Unnamed: 0,Id,Class
0,211,8602,Malignant
1,315,7464,Malignant
2,165,5832,Malignant
3,188,2843,Malignant
4,710,5272,Benign
...,...,...,...
534,763,8176,Benign
535,192,5434,Malignant
536,629,5366,Benign
537,559,9811,Benign


***
#### read the image folder

In [7]:
images = os.listdir(image_folder)

In [8]:
id_to_label_train = dict(zip(train_df['Id'], train_df['Class']))

***
##### read the train folder

In [9]:
# create a list for both images and labels
images_train = []
labels_train = []

In [10]:
for index, row in train_df.iterrows():
  im_id = row['Id']
  if im_id in id_to_label_train :
    label = id_to_label_train [im_id]
    image_path = os.path.join(image_folder, f"{im_id}.png")
    if os.path.exists(image_path):  # Check if the image file exists
        image = cv2.imread(image_path)  # Load the image using cv2

        if image is not None:  # Check if the image was loaded successfully
            images_train.append(image)
            labels_train.append(label)
            print(labels_train)

['Malignant']
['Malignant', 'Malignant']
['Malignant', 'Malignant', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant', 'Benign', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignant', 'Benign', 'Malignant', 'Normal', 'Benign', 'Malignant', 'Benign', 'Benign', 'Benign']
['Malignant', 'Malignant', 'Malignant', 'Malignan

In [163]:
X_train = np.array(images_train)
X_train.shape

<ipython-input-163-8020449d68be>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array(images_train)


(539,)

In [164]:
y_train = np.array(labels_train)
y_train.shape

(539,)

***
##### read the test folder

In [165]:
# Get the image IDs from the test CSV
test_image_ids = list(test_df['Id'])

In [166]:
image_test = []
for index, row in test_df.iterrows():
  image_id = row['Id']
  if image_id in test_image_ids:
    image_path = os.path.join(image_folder, f"{image_id}.png")
    # Check if the image file exists
    if os.path.exists(image_path):
      # Load the image using cv2
      image = cv2.imread(image_path)
      image_test.append(image)





In [167]:
image_test

[array([[[151, 151, 151],
         [ 11,  11,  11],
         [  0,   0,   0],
         ...,
         [169, 169, 169],
         [170, 170, 170],
         [169, 169, 169]],
 
        [[  0,   0,   0],
         [  7,   7,   7],
         [  1,   1,   1],
         ...,
         [152, 152, 152],
         [149, 149, 149],
         [146, 146, 146]],
 
        [[  1,   1,   1],
         [  0,   0,   0],
         [138, 138, 138],
         ...,
         [153, 153, 153],
         [153, 153, 153],
         [152, 152, 152]],
 
        ...,
 
        [[ 48,  48,  48],
         [ 49,  49,  49],
         [ 50,  50,  50],
         ...,
         [ 42,  42,  42],
         [ 42,  42,  42],
         [ 42,  42,  42]],
 
        [[ 45,  45,  45],
         [ 46,  46,  46],
         [ 46,  46,  46],
         ...,
         [ 42,  42,  42],
         [ 44,  44,  44],
         [ 44,  44,  44]],
 
        [[ 43,  43,  43],
         [ 43,  43,  43],
         [ 43,  43,  43],
         ...,
         [ 43,  43,  43],
  

In [168]:
X_test = np.array(image_test)
X_test.shape

<ipython-input-168-71dd8b3a7a86>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.array(image_test)


(229,)

***
know more information about the data


In [169]:
# summarize dataset shape
print('Train', X_train.shape, y_train.shape)
print('Test', (X_test.shape))
# summarize dataset shape
print('Train for image', X_train[0].shape, y_train[0].shape)
print('Test for image', (X_test[0].shape))

Train (539,) (539,)
Test (229,)
Train for image (470, 564, 3) ()
Test for image (467, 551, 3)


In [170]:
type(X_train)

numpy.ndarray

***

to here we will start prepraring data
***


### resize the image

for train

In [171]:
target_size = (200, 200)  # Replace with your desired target size

# Initialize an empty list to store resized images
resized_images = []

# Loop through each image in X_train and resize
for image in X_train:
    resized_image = cv2.resize(image, target_size)
    resized_images.append(resized_image)

# Convert the list of resized images back to a NumPy array
X_train = np.array(resized_images)
# Now X_train_resized contains the resized images
print("Shape of X_train_resized:", X_train.shape)

Shape of X_train_resized: (539, 200, 200, 3)


for test

In [172]:
target_size = (200, 200)  # Replace with your desired target size

# Initialize an empty list to store resized images
resized_images = []

# Loop through each image in X_test and resize
for image in X_test:
    resized_image = cv2.resize(image, target_size)
    resized_images.append(resized_image)

# Convert the list of resized images back to a NumPy array
X_test = np.array(resized_images)
# Now X_test contains the resized images
print("Shape of X_test:", X_test.shape)

Shape of X_test: (229, 200, 200, 3)


***
shape min max std mean for X_train and X_test


In [173]:
# Now compute the minimum and maximum values
minimum_pixel_value_train = np.min(X_train)
maximum_pixel_value_train = np.max(X_train)
print("Train - Minimum Pixel Value:", minimum_pixel_value_train)
print("Train - Maximum Pixel Value:", maximum_pixel_value_train)

minimum_pixel_value_test = np.min(X_test)
maximum_pixel_value_test = np.max(X_test)
print("Test - Minimum Pixel Value:", minimum_pixel_value_test)
print("Test - Maximum Pixel Value:", maximum_pixel_value_test)

Train - Minimum Pixel Value: 0
Train - Maximum Pixel Value: 255
Test - Minimum Pixel Value: 0
Test - Maximum Pixel Value: 255


### split the train into train and val set

In [174]:
# Split data into train and test sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [175]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_train, return_counts=True)
print("Class Frequencies in y_train:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

Class Frequencies in y_train:
Class Benign: 243 samples
Class Malignant: 118 samples
Class Normal: 70 samples


In [176]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_val, return_counts=True)
print("Class Frequencies in y_train:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

Class Frequencies in y_train:
Class Benign: 57 samples
Class Malignant: 30 samples
Class Normal: 21 samples


In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

### data augmentation

In [ ]:
# !pip install Augmentor

In [ ]:
# rotation_range=20,
#     zoom_range=0.15,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     fill_mode="nearest"

In [ ]:
 {'Benign': 0, 'Malignant': 1, 'Normal': 2}

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Define the label encoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

In [ ]:
X_train.shape

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa
import numpy as np

In [ ]:
# Define augmentation transforms
seq = iaa.Sequential([
    iaa.Fliplr(0.2),
    iaa.Affine(rotate=(-2, 2), shear=(-2,20), scale=(0.8, 1.2), mode='reflect'),
    iaa.ElasticTransformation(alpha=(0, 2.0), sigma=0.25),
    iaa.AdditiveGaussianNoise(scale=(0, 0.05 * 255)),
    iaa.GaussianBlur(sigma=(0, 1.0)),
    iaa.AllChannelsHistogramEqualization(),
    iaa.GammaContrast(gamma=(0.2, 1.5)),
    iaa.LinearContrast(alpha=(0.5, 2.0))
], random_order=True)


In [ ]:
X_augmented = []
y_augmented = []

classes = np.unique(y_train)
target_counts = [200, 200,200]

for class_label in classes:
    class_indices = np.where(y_train == class_label)[0]
    num_samples = len(class_indices)
    target_count = target_counts[class_label]

    if num_samples < target_count:
        num_augmentations = target_count - num_samples

        for _ in range(num_augmentations):
            random_index = np.random.choice(class_indices)
            image = X_train[random_index]
            augmented_image = seq.augment_image(image)
            X_augmented.append(augmented_image)
            y_augmented.append(class_label)


In [ ]:
X_augmented = np.array(X_augmented)
y_augmented = np.array(y_augmented)

In [ ]:
X_augmented.shape

In [ ]:
X_train_augmented = np.concatenate([X_train, X_augmented])
y_train_augmented = np.concatenate([y_train, y_augmented])

In [ ]:
class_mapping = {
    0: "Benign",
    1: "Malignant",
    2: "Normal"
}

# Assuming y_train_augmented contains the numerical class labels
y_train_augmented = list(map(lambda label: class_mapping[label], y_train_augmented))
y_train_augmented

In [ ]:
from sklearn.utils import shuffle
X_train_augmented, y_train_augmented = shuffle(X_train_augmented, y_train_augmented, random_state=42)

In [ ]:
y_train_augmented

In [ ]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_train_augmented, return_counts=True)
print("Class Frequencies in y_train:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

In [ ]:
X_train_augmented.shape

In [ ]:
# # Calculate mean HOG feature vectors for each class
# mean_hog_by_class = {}  # Dictionary to store mean HOG vectors for each class
# for class_label in np.unique(y_train):
#     class_samples = X_train[y_train == class_label]
#     mean_hog = np.mean(class_samples, axis=0)
#     mean_hog_by_class[class_label] = mean_hog

# # Generate synthetic samples by adding noise to mean HOG vectors
# num_synthetic_samples = 100  # Adjust as needed
# synthetic_X_train, synthetic_y_train = [], []

# for class_label, mean_hog in mean_hog_by_class.items():
#     for _ in range(num_synthetic_samples):
#         noise = np.random.normal(0, 0.01, mean_hog.shape)  # Adjust noise level
#         synthetic_hog = mean_hog + noise
#         synthetic_X_train.append(synthetic_hog)
#         synthetic_y_train.append(class_label)

# # Convert lists to numpy arrays
# synthetic_X_train = np.array(synthetic_X_train)
# synthetic_y_train = np.array(synthetic_y_train)

# # Concatenate synthetic data with the original data
# final_X_train = np.concatenate((X_train, synthetic_X_train), axis=0)
# final_y_train = np.concatenate((y_train, synthetic_y_train), axis=0)

In [ ]:
# X_aug.shape,y_aug.shape

***
### HOG

train

In [ ]:

# # Apply HOG
# # Set HOG parameters
# orientations = 9
# pixels_per_cell = (8, 8)
# cells_per_block = (2, 2)
# block_norm = 'L2-Hys'

# # Initialize lists to store HOG features
# train_hog_features = []

# # Loop through training images
# for image in X_train_augmented:
#     gray_image = color.rgb2gray(image)
#     features = hog(gray_image, orientations=orientations, pixels_per_cell=pixels_per_cell,
#                    cells_per_block=cells_per_block, block_norm=block_norm)
#     train_hog_features.append(features)

# X_train_hog = np.array(train_hog_features)


In [45]:
# X_train_hog.shape

***
validation

In [46]:
# val_hog_features = []
# # Loop through training images
# for image in X_val:
#     gray_image = color.rgb2gray(image)
#     features = hog(gray_image, orientations=orientations, pixels_per_cell=pixels_per_cell,
#                    cells_per_block=cells_per_block, block_norm=block_norm)
#     val_hog_features.append(features)

# X_val_hog = np.array(val_hog_features)

In [47]:
# X_val_hog.shape


test

In [48]:
# # Loop through test images
# test_hog_features = []

# for image in X_test:
#     gray_image = color.rgb2gray(image)
#     features = hog(gray_image, orientations=orientations, pixels_per_cell=pixels_per_cell,
#                    cells_per_block=cells_per_block, block_norm=block_norm)
#     test_hog_features.append(features)

# # Convert feature lists to numpy arrays
# X_test_hog = np.array(test_hog_features)


***
### filter

In [177]:
import cv2
import numpy as np
from skimage.feature import hog

# Assuming you have loaded X_train, y_train, X_val, y_val, X_test, y_test

# Define the filter (e.g., Gaussian blur)
filter_kernel = np.array([
    [-1, -1, -1],
    [-1, 8, -1],
    [-1, -1, -1]
])   # Normalize the filter kernel

# Apply the filter to images
def apply_filter(images):
    filtered_images = []
    for image in images:
        filtered_image = cv2.filter2D(image, -1, filter_kernel)
        filtered_images.append(filtered_image)
    return np.array(filtered_images)

# Apply the filter to training images
X_filtered_train = apply_filter(X_train_augmented)

# Apply the filter to validation images
X_filtered_val = apply_filter(X_val)

# Apply the filter to test images
X_filtered_test = apply_filter(X_test)

# Convert color images to grayscale (assuming 3 channels in each image)
X_filtered_train_gray = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X_filtered_train])
X_filtered_val_gray = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X_filtered_val])
X_filtered_test_gray = np.array([cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in X_filtered_test])

# Define HOG parameters
hog_params = {
    'orientations': 9,
    'pixels_per_cell': (8, 8),
    'cells_per_block': (2, 2),
    'transform_sqrt': True,
    'visualize': False
}

# Compute HOG features
def compute_hog(images):
    hog_features = []
    for image in images:
        hog_feature = hog(image, **hog_params)
        hog_features.append(hog_feature)
    return np.array(hog_features)

# Compute HOG features for filtered grayscale training images
X_train_hog_filtered = compute_hog(X_filtered_train_gray)

# Compute HOG features for filtered grayscale validation images
X_val_hog_filtered = compute_hog(X_filtered_val_gray)

# Compute HOG features for filtered grayscale test images
X_test_hog_filtered = compute_hog(X_filtered_test_gray)


In [178]:
X_train_hog_filtered.shape

(431, 20736)

In [179]:
X_train_hog_filtered

array([[0.28227361, 0.07436073, 0.21312034, ..., 0.09661175, 0.19275571,
        0.05939918],
       [0.35934606, 0.0554641 , 0.        , ..., 0.04331133, 0.16149199,
        0.        ],
       [0.25635601, 0.18414039, 0.14182784, ..., 0.02373937, 0.        ,
        0.01930437],
       ...,
       [0.28179305, 0.06241702, 0.20725813, ..., 0.08993566, 0.10409771,
        0.02886102],
       [0.26881166, 0.09031793, 0.26881166, ..., 0.20473297, 0.21282793,
        0.04547952],
       [0.28907379, 0.17151944, 0.28907379, ..., 0.15453086, 0.08223265,
        0.03154572]])

In [180]:
X_train_augmented.shape

(643, 200, 200, 3)

In [181]:
X_val.shape

(108, 200, 200, 3)

In [182]:
X_val_hog_filtered.shape

(108, 20736)

In [190]:
X_train_final =  X_train_hog_filtered
X_val_final = X_val_hog_filtered
X_test_final = X_test_hog_filtered

y_train_final = y_train  # Use the appropriate labels
y_val_final = y_val# Use the appropriate labels
  # Use the appropriate labels


In [149]:
X_train = X_train_final
X_val = X_val_final
X_test = X_test_final

In [150]:
y_val_final.shape

(108,)

In [151]:
X_train_final.shape,X_val_final.shape

((643, 20736), (108, 20736))

In [152]:
# # Convert feature list to numpy array
# X_hog = np.array(X_train_hog)

### normalizing pixel value the image

train

In [185]:
X_train = X_train_final / 255.0
X_train[0]

array([0.00110696, 0.00029161, 0.00083577, ..., 0.00037887, 0.0007559 ,
       0.00023294])

test

In [186]:
X_val = X_val_final / 255.0
X_val[0]

array([1.06438647e-03, 1.81042948e-04, 7.60919544e-04, ...,
       4.46049397e-05, 4.45919441e-04, 2.28023769e-04])

In [187]:
X_test = X_test_final / 255.0
X_test[0]

array([1.01396787e-03, 3.06166481e-04, 8.21600005e-04, ...,
       0.00000000e+00, 9.80780708e-05, 0.00000000e+00])

In [184]:
# Now compute the minimum and maximum values
minimum_pixel_value_train = np.min(X_train)
maximum_pixel_value_train = np.max(X_train)
print("Train - Minimum Pixel Value:", minimum_pixel_value_train)
print("Train - Maximum Pixel Value:", maximum_pixel_value_train)

minimum_pixel_value_test = np.min(X_test)
maximum_pixel_value_test = np.max(X_test)
print("Test - Minimum Pixel Value:", minimum_pixel_value_test)
print("Test - Maximum Pixel Value:", maximum_pixel_value_test)

Train - Minimum Pixel Value: 0
Train - Maximum Pixel Value: 255
Test - Minimum Pixel Value: 0
Test - Maximum Pixel Value: 255


***
### Flaten
train

In [67]:
# # Flatten the HOG features using the flatten() function
# X_train = X_train.flatten()
# X_train.shape

(13333248,)

test

In [68]:
# # Flatten the HOG features using the flatten() function
# X_val = X_val.flatten()
# X_val.shape

(2239488,)

In [65]:
# # Flatten the HOG features using the flatten() function
# X_test = X_test_hog.flatten()
# X_test.shape

In [75]:
# # Reshape the input data to 2D array
# X_train = X_train.reshape(-1, 1)
# X_val = X_val.reshape(-1, 1)


In [157]:
X_train.shape,X_val.shape

((643, 20736), (108, 20736))

### model

In [63]:
from sklearn.preprocessing import LabelEncoder
# Define the label encoder
label_encoder = LabelEncoder()
y_train_final = label_encoder.fit_transform(y_train_final)
y_val_final = label_encoder.transform(y_val_final)

In [78]:
y_val_final.shape

(108,)

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [85]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [81]:
# # Create a Random Forest classifier
# rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# # Train the model
# rf_model.fit(X_train, y_train_final)

# # Predict on validation data
# y_pred = rf_model.predict(X_val)

# # Print classification report
# class_names = ['Class 0', 'Class 1', 'Class 2']
# classification_rep = classification_report(y_val_final, y_pred, target_names=class_names)
# print("Classification Report:")
# print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.57      0.89      0.69        57
     Class 1       0.77      0.33      0.47        30
     Class 2       0.20      0.05      0.08        21

    accuracy                           0.57       108
   macro avg       0.51      0.43      0.41       108
weighted avg       0.55      0.57      0.51       108



Classification Report:
              precision    recall  f1-score   support

     Class 0       0.57      0.89      0.69        57
     Class 1       0.77      0.33      0.47        30
     Class 2       0.20      0.05      0.08        21

    accuracy                           0.57       108
   macro avg       0.51      0.43      0.41       108
weighted avg       0.55      0.57      0.51       108

In [68]:
!pip install catboost

Benign: 250 samples
Class Malignant: 250 samples
Class Normal

In [69]:
import xgboost as xgb
from sklearn.metrics import classification_report

In [ ]:
# # Define XGBoost parameters
# params = {
#     'objective': 'multi:softmax',  # Multi-class classification
#     'num_class': 3,  # Number of classes
#     'tree_method': 'gpu_hist',  # Enable GPU histogram method
#     'gpu_id': 0 ,
#     'learning_rate':0.1# Specify GPU device index
# }

# # Create DMatrix for training and validation data
# dtrain = xgb.DMatrix(X_train, label=y_train_final)
# dval = xgb.DMatrix(X_val, label=y_val_final)

# # Train the XGBoost model
# model = xgb.train(params, dtrain)

# # Predict on validation data
# y_pred = model.predict(dval)

# # Print classification report
# class_names = ['Class 0', 'Class 1', 'Class 2']
# classification_rep = classification_report(y_val_final, y_pred, target_names=class_names)
# print("Classification Report:")
# print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.57      0.75      0.65        57
     Class 1       0.43      0.33      0.38        30
     Class 2       0.33      0.14      0.20        21

    accuracy                           0.52       108
   macro avg       0.44      0.41      0.41       108
weighted avg       0.48      0.52      0.48       108


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from catboost import CatBoostClassifier
# from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# # Define your CatBoost parameters
# catboost_params = {
#     'iterations': 5000,
#     'learning_rate': 0.1,
#     'depth': 6,
#     'loss_function': 'MultiClass',
#     'task_type': 'GPU',  # Use GPU for training
#     'devices': '0',  # Specify GPU device index
#     'verbose': 100
# }

# # Create the CatBoostClassifier
# model = CatBoostClassifier(**catboost_params)

# # Train the model on your filtered and HOG-processed training data
# model.fit(X_train, y_train_final, early_stopping_rounds=50)

# # # Plot learning curves
# # iterations = np.arange(0, model.tree_count_, model.verbose)
# # plt.plot(iterations, model.evals_result_['learn']['MultiClass'], label='Train')
# # plt.plot(iterations, model.evals_result_['validation']['MultiClass'], label='Validation')
# # plt.xlabel('Iterations')
# # plt.ylabel('Logloss')
# # plt.title('CatBoost Learning Curves')
# # plt.legend()
# # plt.show()

# # Predict on test data
# y_pred = model.predict(X_val)

# # Calculate accuracy
# accuracy = accuracy_score(y_val_final, y_pred)
# print(f"Test Accuracy: {accuracy:.2f}")

# # Generate classification report
# class_names = ['Class 0', 'Class 1', 'Class 2']
# classification_rep = classification_report(y_val_final, y_pred, target_names=class_names)
# print("Classification Report:")
# print(classification_rep)

# # Generate confusion matrix
# conf_matrix = confusion_matrix(y_val_final, y_pred)
# print("Confusion Matrix:")
# print(conf_matrix)


In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from catboost import CatBoostClassifier
# from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# # Define your CatBoost parameters
# catboost_params = {
#     'iterations': 1000,
#     'learning_rate': 0.1,
#     'depth': 8,
#     'loss_function': 'MultiClass',
#     'task_type': 'GPU',  # Use GPU for training
#     'devices': '0',  # Specify GPU device index
#     'verbose': 100
# }

# # Create the CatBoostClassifier
# model = CatBoostClassifier(**catboost_params)

# # Train the model on your filtered and HOG-processed training data
# model.fit(X_train, y_train_final, early_stopping_rounds=50)

# # # Plot learning curves
# # iterations = np.arange(0, model.tree_count_, model.verbose)
# # plt.plot(iterations, model.evals_result_['learn']['MultiClass'], label='Train')
# # plt.plot(iterations, model.evals_result_['validation']['MultiClass'], label='Validation')
# # plt.xlabel('Iterations')
# # plt.ylabel('Logloss')
# # plt.title('CatBoost Learning Curves')
# # plt.legend()
# # plt.show()

# # Predict on test data
# y_pred = model.predict(X_val)

# # Calculate accuracy
# accuracy = accuracy_score(y_val_final, y_pred)
# print(f"Test Accuracy: {accuracy:.2f}")

# # Generate classification report
# class_names = ['Class 0', 'Class 1', 'Class 2']
# classification_rep = classification_report(y_val_final, y_pred, target_names=class_names)
# print("Classification Report:")
# print(classification_rep)

# # Generate confusion matrix
# conf_matrix = confusion_matrix(y_val_final, y_pred)
# print("Confusion Matrix:")
# print(conf_matrix)


In [191]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

svm_classifier = SVC(kernel='rbf', C=100,gamma= 0.001)
svm_classifier.fit(X_train, y_train)

# Make predictions
y_pred = svm_classifier.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val_final, y_pred)
print("Accuracy:", accuracy)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_val_final, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Generate classification report
class_report = classification_report(y_val_final, y_pred)
print("Classification Report:\n", class_report)

Accuracy: 0.5277777777777778
Confusion Matrix:
 [[57  0  0]
 [30  0  0]
 [21  0  0]]
Classification Report:
               precision    recall  f1-score   support

      Benign       0.53      1.00      0.69        57
   Malignant       0.00      0.00      0.00        30
      Normal       0.00      0.00      0.00        21

    accuracy                           0.53       108
   macro avg       0.18      0.33      0.23       108
weighted avg       0.28      0.53      0.36       108



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [160]:
# Define the parameter grid for grid search
param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']}

# Create an SVM classifier
svm_classifier = SVC()

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, scoring='accuracy', cv=5)

# Fit the grid search to the data
grid_search.fit(X_train, y_train_final)

# Get the best parameters and best estimator from the grid search
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print("Best Parameters:", best_params)

Best Parameters: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}


Best Parameters: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}


In [161]:
# Fit the best estimator on the training data
best_estimator.fit(X_train, y_train_final)

# Make predictions on the validation set
y_pred = best_estimator.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val_final, y_pred)
print("Accuracy:", accuracy)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_val_final, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Generate classification report
class_report = classification_report(y_val_final, y_pred)
print("Classification Report:\n", class_report)

Accuracy: 0.6018518518518519
Confusion Matrix:
 [[47  6  4]
 [16 12  2]
 [15  0  6]]
Classification Report:
               precision    recall  f1-score   support

      Benign       0.60      0.82      0.70        57
   Malignant       0.67      0.40      0.50        30
      Normal       0.50      0.29      0.36        21

    accuracy                           0.60       108
   macro avg       0.59      0.50      0.52       108
weighted avg       0.60      0.60      0.58       108



Accuracy: 0.8869047619047619
Confusion Matrix:
 [[73  2  0]
 [ 9 43  1]
 [ 7  0 33]]
Classification Report:
               precision    recall  f1-score   support

      Benign       0.82      0.97      0.89        75
   Malignant       0.96      0.81      0.88        53
      Normal       0.97      0.82      0.89        40

    accuracy                           0.89       168
   macro avg       0.92      0.87      0.89       168
weighted avg       0.90      0.89      0.89       168

In [ ]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_val, return_counts=True)
print("Class Frequencies in y_val:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

In [ ]:
# Calculate class frequencies for y_test
unique_classes, class_counts_test = np.unique(y_pred, return_counts=True)
print("Class Frequencies in y_val:")
for cls, count in zip(unique_classes, class_counts_test):
    print(f"Class {cls}: {count} samples")

### test now

In [311]:
# Make predictions
predictions = model.predict(X_test)

In [312]:
test_df['labels'] = predictions
test_df['labels'].value_counts()

Benign       175
Malignant     44
Normal        10
Name: labels, dtype: int64

In [313]:
submit = pd.read_csv('/content/sample_submition4.csv')
submit

,Id,Class
0,8560,A
1,7920,A
2,8993,A
3,8126,A
4,9912,A
...,...,...
224,8343,A
225,1255,A
226,4573,A
227,3919,A


In [314]:
submit['Id'] = test_df['Id']
submit['Class'] =predictions # our model predictions on the test dataset
submit

,Id,Class
0,8560,Benign
1,7920,Benign
2,8993,Malignant
3,8126,Benign
4,9912,Benign
...,...,...
224,8343,Benign
225,1255,Benign
226,4573,Benign
227,3919,Benign


In [315]:
submit.to_csv('sample_submission_eman_new_way_try_filter3.csv', index=False)